In [1]:
import pandas as pd

In [2]:
df_titanic = pd.read_csv("data/traintitanic.csv")

In [3]:
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
import atoti as tt

session = tt.create_session()

In [5]:
df_titanic_ss = session.read_csv("data/traintitanic.csv", keys=["PassengerId"])

In [26]:
type(df_titanic_ss)

atoti.store.Store

In [27]:

df_titanic_ss.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,N/A,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,N/A,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,N/A,S


In [28]:
titanic_cube = session.create_cube(df_titanic_ss, "titanic")

In [29]:
?session.create_cube

Signature:
session.create_cube(
    base_store: 'Store',
    name: 'Optional[str]' = None,
    *,
    mode: '_CubeCreationMode' = 'auto',
) -> 'Cube'
Docstring:
Create a cube based on the passed store.

Args:
    base_store: The base store of the cube.
    name: The name of the created cube.
        Defaults to the name of the base store.
    mode: The cube creation mode:

        * ``auto``: Creates hierarchies for every non-numeric column, and measures for every numeric column.
        * ``manual``: Does not create any hierarchy or measure (except from the count).
        * ``no_measures``: Creates the hierarchies like ``auto`` but does not create any measures.

        For stores with ``hierarchized_columns`` specified, these will be converted into
        hierarchies regardless of the cube creation mode.

See Also:
    Hierarchies and measures created by a :meth:`~atoti.store.Store.join`.
File:      ~/miniconda3/envs/atoti/lib/python3.8/site-packages/atoti/session.py
Type:      met

In [30]:
titanic_cube.hierarchies

Hierarchies()

In [31]:
m = titanic_cube.measures
m

Measures()

In [32]:
l = titanic_cube.levels
l

Levels()

In [13]:
titanic_cube.query(m["Age.MEAN"])

,Age.MEAN
0,29.70


In [14]:
titanic_cube.query(m["Age.MEAN"], levels=l["Sex"])

,Age.MEAN
Sex,
female,27.92
male,30.73


In [66]:
mdx_query = """
SELECT {
[Measures].[Age.MEAN], 
[Measures].[Fare.MEAN]
} ON COLUMNS, 
{[Traintitanic].[Sex]} ON ROWS
FROM [titanic]
"""

In [93]:
mdx_query2 = """
SELECT {
[Measures].[Age.MEAN], 
[Measures].[Fare.MEAN]
} ON COLUMNS, 
 NON EMPTY Crossjoin(
{[Traintitanic].[Sex].[Sex].Members, 
[Traintitanic].[Survived].[Survived].Members}) ON ROWS
FROM [titanic]
"""

In [89]:


mdx = """SELECT
  {
    [Measures].[Price.AVG],
    [Measures].[Quantity.SUM]
  } ON COLUMNS,
  NON EMPTY Crossjoin(
    [Hierarchies].[Color].[Color].Members,
    [Hierarchies].[Continent].[Continent].Members
  ) ON ROWS
  FROM [Cube]
  """

In [90]:
titanic_cube

Cube(_name='titanic', _hierarchies=Hierarchies(), _measures=Measures())

In [91]:
session.query_mdx(mdx_query)

,Age.MEAN,Fare.MEAN
0,29.70,32.20


In [94]:
session.query_mdx(mdx_query2)

RuntimeError: ('Request failed', {'status': 'error', 'error': {'errorChain': [{'type': 'MdxException', 'message': 'No matching signature for Crossjoin({[Traintitanic].[Sex].[Sex].Members, [Traintitanic].[Survived].[Survived].Members}). Available signatures are: Crossjoin(<Set> ...)'}, {'type': 'MdxException', 'message': 'Failed resolution of expression: Crossjoin({[Traintitanic].[Sex].[Sex].Members, [Traintitanic].[Survived].[Survived].Members})'}, {'type': 'MdxException', 'message': ' SELECT { [Measures].[Age.MEAN],  [Measures].[Fare.MEAN] } ON COLUMNS,   NON EMPTY Crossjoin( {[Traintitanic].[Sex].[Sex].Members,  [Traintitanic].[Survived].[Survived].Members}) ON ROWS FROM [titanic] '}, {'type': 'QueryException', 'message': 'ActivePivotSyncActionQuery: action=MdxAction (mdx= SELECT { [Measures].[Age.MEAN],  [Measures].[Fare.MEAN] } ON COLUMNS,   NON EMPTY Crossjoin( {[Traintitanic].[Sex].[Sex].Members,  [Traintitanic].[Survived].[Survived].Members}) ON ROWS FROM [titanic] ), argument=c